# Getting Started with Langchain Examples

2025.02.19 

https://www.langchain.com/
https://python.langchain.com/docs/tutorials/


## Getting Started with Chatbots
https://python.langchain.com/docs/tutorials/llm_chain/

### Get OpenAI API KEY from .env file

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# The API key will be automatically loaded from .env
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [3]:
# Show model object class structure
#dir(model)

Let's first use the model directly. ChatModels are instances of LangChain Runnables, which means they expose a standard interface for interacting with them. To simply call the model, we can pass in a list of messages to the .invoke method.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("Welcome to Langchain Nicole, Dustin, Maxwell, and Rocio!"),
]

translation_response = model.invoke(messages)
translation_response

# Parsing LangChain Response

## Part 1: Capturing Content Response


In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
import json

# Your existing messages
messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("Welcome to Langchain Nicole, Dustin, Maxwell, and Rocio!"),
]

translation_response = model.invoke(messages)
translation_response

AIMessage(content='Benvenuti a Langchain Nicole, Dustin, Maxwell e Rocio!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 32, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'stop', 'logprobs': None}, id='run-5d595434-d678-42cb-b790-3cd0410280c9-0', usage_metadata={'input_tokens': 32, 'output_tokens': 16, 'total_tokens': 48, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Extract just the `content`


In [7]:
content = translation_response.content
print("Translated Content:")
print(content)

Translated Content:
Benvenuti a Langchain Nicole, Dustin, Maxwell e Rocio!




## Part 2: Pretty Print Full Response


In [10]:
from pprint import pprint

# Pretty print the full response object
print("\nFull Response Details:")
pprint(vars(translation_response))



Full Response Details:
{'additional_kwargs': {'refusal': None},
 'content': 'Benvenuti a Langchain Nicole, Dustin, Maxwell e Rocio!',
 'example': False,
 'id': 'run-5d595434-d678-42cb-b790-3cd0410280c9-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'stop',
                       'logprobs': None,
                       'model_name': 'gpt-4o-mini-2024-07-18',
                       'system_fingerprint': 'fp_00428b782a',
                       'token_usage': {'completion_tokens': 16,
                                       'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                     'audio_tokens': 0,
                                                                     'reasoning_tokens': 0,
                                                                     'rejected_prediction_tokens': 0},
                                       'prompt_tokens': 32,
                            

### Alternative method using json

This will:
1. First extract and display just the translated content
2. Then show all response properties in a formatted way
3. Finally display a JSON-formatted version of the complete response

The `pprint` and JSON formatting make it easier to read all the metadata and additional properties that come with the AIMessage response.


In [ ]:
print("\nJSON Formatted Response:")
response_dict = {
    "type": translation_response.type,
    "content": translation_response.content,
    "additional_kwargs": translation_response.additional_kwargs,
    "example": translation_response.example
}
print(json.dumps(response_dict, indent=2))

# Stream Handling of Messages

Because chat models are Runnables, they expose a standard interface that includes async and streaming modes of invocation. This allows us to stream individual tokens from a chat model:

Using the messages from earlier, we can see how this works...

In [12]:
messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("Welcome to Langchain Nicole, Dustin, Maxwell, and Rocio!"),
]

In [14]:
for token in model.stream(messages):
    print(token.content, end="| ")

| Ben| ven| uti|  a|  Lang| chain|  Nicole| ,|  Dustin| ,|  Maxwell|  e|  Roc| io| !| | 

# Prompt Templates

Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

Prompt templates [https://python.langchain.com/docs/concepts/prompt_templates/] are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

Let's create a prompt template here. It will take in two user variables:

    language: The language to translate text into
    text: The text to translate


In [16]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [19]:
#help(ChatPromptTemplate)

system_template

'Translate the following from English into {language}'

In [20]:
prompt_template

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following from English into {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

Note that ChatPromptTemplate supports multiple message roles in a single template. We format the language parameter into the system message, and the user text into a user message.

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself.

In [21]:
prompt = prompt_template.invoke({"language": "Italian", "text": "Hey Giacomo!  Check out Langchain.  The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself."})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hey Giacomo!  Check out Langchain.  The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself.', additional_kwargs={}, response_metadata={})])

In [22]:
Giacomo_langchain_translation_response = model.invoke(prompt)
Giacomo_langchain_translation_response

AIMessage(content="Ciao Giacomo! Dai un'occhiata a Langchain. L'input di questo modello di prompt è un dizionario. Possiamo sperimentare con questo modello di prompt per vedere cosa fa da solo.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 58, 'total_tokens': 104, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac292ba1-3aa5-4632-a1da-d9c42aa18047-0', usage_metadata={'input_tokens': 58, 'output_tokens': 46, 'total_tokens': 104, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

<<< RICH LEFT OFF HERE - 2025.02.19 10:35PM>>>